In [4]:
import urllib.request
import datetime
import csv
import time
import random
import re
from bs4 import BeautifulSoup

In [5]:


web_address = 'https://www.presidency.ucsb.edu/documents/app-categories/presidential/spoken-addresses-and-remarks?items_per_page=60&page='
web_aux = 'https://www.presidency.ucsb.edu'
date_start = datetime.datetime(2021, 1, 20, 0, 0)  # date in which Biden's term starts
speech = {}
page_number = 0

with open('pres_speech.csv', 'w', newline='', encoding='utf-8') as f:
    w = csv.DictWriter(f, fieldnames=["date", "title", "content", "citation", "footnote"])  # define the columns name
    w.writeheader()

    while True:  # Use a while loop to handle pagination
        try:
            web_page = urllib.request.urlopen(web_address + str(page_number))
            soup = BeautifulSoup(web_page, 'html.parser')
            all_date_speech = soup.find_all(class_=re.compile("col-sm-8"))

            for i in range(len(all_date_speech)):  # Loop through posts on the page
                date_speech_element = all_date_speech[i].find('span')
                if date_speech_element:
                    speech['date'] = date_speech_element.attrs['content'][:10]  # Get the date and cut the time
                    speech['date'] = datetime.datetime.strptime(speech['date'], '%Y-%m-%d')
                    
                    if speech['date'] < date_start:
                        print("Reached speeches before Biden's presidency.")
                        break

                    inner_page_url = web_aux + all_date_speech[i].find('a').attrs['href']
                    inner_page = urllib.request.urlopen(inner_page_url)
                    inner_soup = BeautifulSoup(inner_page, 'html.parser')

                    speech['title'] = inner_soup.find('h1').text if inner_soup.find('h1') else ''
                    speech['content'] = inner_soup.find('div', {'class': 'field-docs-content'}).text if inner_soup.find('div', {'class': 'field-docs-content'}) else ''
                    speech['footnote'] = inner_soup.find('div', {'class': 'field-docs-footnote'}).text if inner_soup.find('div', {'class': 'field-docs-footnote'}) else ''
                    speech['citation'] = inner_soup.find('div', {'class': 'field-prez-document-citation'}).text if inner_soup.find('div', {'class': 'field-prez-document-citation'}) else ''

                    w.writerow(speech)
                    time.sleep(2)  

            page_number += 1

            if speech['date'] < date_start:
                break  # Stop the loop if we've reached dates before Biden's presidency

        except Exception as e:
            print(f"An error occurred: {e}")
            page_number += 1  # Ensure we move to the next page even if there's an error
            continue  # Continue to the next iteration


Reached speeches before Biden's presidency.


hello
